# Intro to Gragh Convolutional Surrogate Models
Eamon Whalen

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

sys.path.append('./models')
from feastnetSurrogateModel import FeaStNet

sys.path.append('./readers')
from loadGhGraphs import loadGhGraphs

sys.path.append('./visualization')
from altTrussViz import plotTruss, interactiveErrorPlot

sys.path.append('./util')
from gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
doeFile = "/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/design_7_N_1000.csv"
allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.199516
std,2.524026
min,0.006570
25%,0.015425
50%,0.024334
75%,0.046981
max,67.325867


In [3]:
plotTruss(allGraphsUnfiltered[0], showDeformed=True, defScale=100)

alt.LayerChart(...)

## 2. Filter and partition

In [4]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.028952
std,0.019071
min,0.006570
25%,0.014951
50%,0.021719
75%,0.038367
max,0.097861


## 3. Train a GCN

In [5]:
gcn = FeaStNet()
history = gcn.trainModel(trainData, valData, 
                     epochs=100, 
                     batch_size=256, 
                     flatten=True, 
                     logTrans=False, 
                     ssTrans=True, 
                     saveDir='./results/debugging/')

plotHistory(history)

epoch: 0   trainLoss: 9.5878e-01   valLoss:9.7925e-01  time: 1.38e+00
epoch: 1   trainLoss: 9.1731e-01   valLoss:9.7205e-01  time: 1.06e+00
epoch: 2   trainLoss: 9.1801e-01   valLoss:9.4761e-01  time: 1.11e+00
epoch: 3   trainLoss: 8.7190e-01   valLoss:9.3355e-01  time: 1.06e+00
epoch: 4   trainLoss: 8.0284e-01   valLoss:9.3964e-01  time: 1.03e+00
epoch: 5   trainLoss: 7.5724e-01   valLoss:9.3411e-01  time: 1.02e+00
epoch: 6   trainLoss: 7.5585e-01   valLoss:8.9770e-01  time: 1.04e+00
epoch: 7   trainLoss: 7.3866e-01   valLoss:8.4447e-01  time: 1.17e+00
epoch: 8   trainLoss: 6.6083e-01   valLoss:7.6445e-01  time: 1.04e+00
epoch: 9   trainLoss: 7.0764e-01   valLoss:7.0523e-01  time: 1.04e+00
epoch: 10   trainLoss: 6.2778e-01   valLoss:6.7054e-01  time: 1.10e+00
epoch: 11   trainLoss: 6.7680e-01   valLoss:6.5072e-01  time: 1.03e+00
epoch: 12   trainLoss: 6.3979e-01   valLoss:6.2726e-01  time: 1.04e+00
epoch: 13   trainLoss: 5.4419e-01   valLoss:6.0984e-01  time: 1.07e+00
epoch: 14   trai

alt.Chart(...)

## 4. Test the GCN

In [6]:
trainRes = gcn.testModel(trainData)
testRes = gcn.testModel(testData)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000024,0.003338,0.144460,0.851332,0.931309,-0.002070,-11.781241
test,0.000030,0.003571,0.154488,0.662072,0.878537,0.094829,-9.511590


## 5. Visualize some predictions

In [7]:
i = 9
pred = gcn.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=100, prediction=pred)

alt.LayerChart(...)

In [8]:
alt.data_transformers.enable('json')
allPreds = gcn.predict(testData)
interactiveErrorPlot(testData, allPreds)

alt.HConcatChart(...)